In [1]:
import os
import time
import pandas as pd

In [6]:
# bad_pipes = ['typical service connection']
# good_pipes = ['extend copper service pipe']

bad_pipes = ['galvanize', 'lead', 'asbest']
good_pipes = ['copper', 'polyvinyl', 'pvc', 'polyethylene', 'hde', 'poly']


text_files_path = r'C:\Temp\all_asbuilt_text_files'
pipe_info = []
start_time = time.time()

# iterate each file in a directory
txt_counter = 0
for file in os.listdir(text_files_path):
    if file.lower().endswith(".txt"):
        asbuilt_name = ""
        title_splitted = file.split(',')
        if len(title_splitted) > 1:
            temp_asbuilt_name = title_splitted[:-1]            
            asbuilt_page = title_splitted[-1].lstrip().rstrip().replace("    ", " ").replace("   ", " ").replace("  ", " ")[:-4]
            asbuilt_name = asbuilt_name.join(temp_asbuilt_name)

            if "page" not in asbuilt_page.lower():
                asbuilt_page = "Page 1"
        else:
            temp_asbuilt_name = file[:-4]
            asbuilt_name = asbuilt_name.join(temp_asbuilt_name)
            asbuilt_page = "Page 1"

        txt_counter += 1
        cur_path = os.path.join(text_files_path, file)
        with open(cur_path, 'r') as f:
            content = f.read().lower()

        # read all content of a file and search string
        pipe_type = 'Unknown'
        for gp in good_pipes:
            if gp in content:
                # print('{} -- good pipe found on -- {}'.format(gp, file))
                pipe_type = 'Good'
                pipe_data = {
                    "AsBuilt_Name" : asbuilt_name,                                    
                    "AsBuilt_Page" : asbuilt_page,
                    "Pipe_type" : pipe_type,
                    "Keyword" : gp
                }
                pipe_info.append(pipe_data)

        # read all content of a file and search string
        for bp in bad_pipes:
            if bp in content:
                # print('{} -- bad pipe found on -- {}'.format(bp, file))
                pipe_type = 'Bad'
                pipe_data = {
                    "AsBuilt_Name" : asbuilt_name,                                    
                    "AsBuilt_Page" : asbuilt_page,
                    "Pipe_type" : pipe_type,
                    "Keyword" : bp
                }
                pipe_info.append(pipe_data)

        if pipe_type == 'Unknown':
            pipe_data = {
                "AsBuilt_Name" : asbuilt_name,                                    
                "AsBuilt_Page" : asbuilt_page,
                "Pipe_type" : pipe_type,
                "Keyword" : "Unmatched"
            }
            pipe_info.append(pipe_data)

        f.close()

        
print("Total text processed is - {} - ".format(txt_counter))
end_time = time.time()
execution_time = round(end_time-start_time, 2)

keywords_df = pd.DataFrame(pipe_info)
keywords_df.to_csv(os.path.join(dir_path, 'csv_files' , 'asbuilts_pipe_types.csv'))

print("Total processing time : {} ".format(execution_time))

Total text processed is - 105824 - 
Total processing time : 271.99 


In [7]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

Number of processors:  8


In [ ]:
# Parallelizing using Pool.apply()
# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count())

# Step 2: `pool.apply` the `howmany_within_range()`
results = [pool.apply(howmany_within_range, args=(row, 4, 8)) for row in data]

# Step 3: Don't forget to close
pool.close()    

print(results[:10])
#> [3, 1, 4, 4, 4, 2, 1, 1, 3, 3]

In [ ]:
# Parallelizing using Pool.map()

# Redefine, with only 1 mandatory argument.
def howmany_within_range_rowonly(row, minimum=4, maximum=8):
    count = 0
    for n in row:
        if minimum <= n <= maximum:
            count = count + 1
    return count

pool = mp.Pool(mp.cpu_count())

results = pool.map(howmany_within_range_rowonly, [row for row in data])

pool.close()

print(results[:10])

In [ ]:
from multiprocessing import Pool
pool = Pool()
result1 = pool.apply_async(solve1, [A])    # evaluate "solve1(A)" asynchronously
result2 = pool.apply_async(solve2, [B])    # evaluate "solve2(B)" asynchronously
answer1 = result1.get(timeout=10)
answer2 = result2.get(timeout=10)